### **Binary Features: Flag, Bool, True-False**

In [17]:
##################### GEÇMİŞ TANIMLAMALAR ##############################

# Kütüphaneler
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler


# Pandas ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


# Veri setleri
def load_application_train():
    data = pd.read_csv('../01-outliers/application_train.csv')
    return data

def load():
    data = pd.read_csv('../01-outliers/titanic.csv')
    return data

In [18]:
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [19]:
# Cabin değişkeninin null değerlerine 0 diğerlerinin değerleri 1 olan NEW_CABIN_BOOL değişkenini oluşturalım.
 
df["NEW_CABIN_BOOL"] = df["Cabin"].notnull().astype("int64")

In [20]:
# Şimdi NEW_CABIN_BOOL değişkenine göre group by işlemi yapıp kabini belirli olanlarla kabini belirsiz olanların hayatta kalma oranlarına bakalım.
df.groupby("NEW_CABIN_BOOL").agg({"Survived": "mean"}) # pandas.core.groupby.DataFrame

# df.groupby("NEW_CABIN_BOOL")["Survived"].mean() # pandas.core.series.Series



,Survived
NEW_CABIN_BOOL,
0,0.300
1,0.667


In [25]:
# İki grup arasında gerçekten de anlamlı bir ilişki var mı bunun için "Tek Örneklem Oran Testi" yapalım.
from statsmodels.stats.proportion import proportions_ztest

test_stat, pvalue = proportions_ztest(count = [df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum(), df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()],
                                       nobs = [df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0], df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]])

print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

# p-value değeri 0.05 ten düşük ise anlamlı bir ilişki vardır çıkarımı yapabiliriz.

Test Stat = 9.4597, p-value = 0.0000


In [26]:
# Çıkarım yapalım. Yolcunun yalnız olup olmadığını belirleyelim.

df.loc[((df["SibSp"] + df["Parch"]) > 0), "NEW_IS_ALONE"] = "NO"
df.loc[((df["SibSp"] + df["Parch"]) == 0), "NEW_IS_ALONE"] = "YES"

In [27]:
# Yalnız olanların yalnız olmayanlara göre hayatta kalma oranı inceleyelim.
df.groupby("NEW_IS_ALONE").agg({"Survived": "mean"})

,Survived
NEW_IS_ALONE,
NO,0.506
YES,0.304


In [29]:
# İki grup arasında gerçekten de anlamlı bir ilişki var mı bunun için "Tek Örneklem Oran Testi" yapalım.
from statsmodels.stats.proportion import proportions_ztest

test_stat, pvalue = proportions_ztest(count = [df.loc[df["NEW_IS_ALONE"] == 'YES', "Survived"].sum(), df.loc[df["NEW_IS_ALONE"] == 'NO', "Survived"].sum()],
                                       nobs = [df.loc[df["NEW_IS_ALONE"] == 'YES', "Survived"].shape[0], df.loc[df["NEW_IS_ALONE"] == 'NO', "Survived"].shape[0]])

print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

# p-value değeri 0.05 ten düşük ise anlamlı bir ilişki vardır çıkarımı yapabiliriz.

Test Stat = -6.0704, p-value = 0.0000
